In [47]:
import pandas as pd
import random as rd
import datetime as dt
import pickle,glob,re

In [48]:
cumule=True
repNom="/Users/gilles/Box Sync/2015-Data/Samples/"
repNomSortie="/Users/gilles/Box Sync/2015-Data/DerivationParadigmes/"
repNomSortie="/Users/gilles/Box Sync/2015-Data/TEMP/"

In [49]:
listeTirages=glob.glob(repNom+"*-Tirage.pkl")

In [50]:
def tiragesNom(nom):
    result=[tirage for tirage in listeTirages if nom in tirage]
    return result

In [51]:
compositionLongitudinale=[("1Ko",10),("10Ko",9),("100Ko",9),("1Mo",19)]
compositionLongitudinale=[("10Mo",20)]

In [52]:
serieLongitudinale=[]
for serie,nbSerie in compositionLongitudinale:
    for nom in rd.sample(tiragesNom(serie),nbSerie):
        m=re.match(ur"^.*-\d{6}-%s-(\d{2})-.*"%serie,nom)
        if m:
#            print m.group(1), nom
            serieLongitudinale.append(nom)
        else:
            print "mauvais nom de fichier",nom
serieLongitudinale

['/Users/gilles/Box Sync/2015-Data/Samples/MGC-170506-10Mo-45-Tirage.pkl',
 '/Users/gilles/Box Sync/2015-Data/Samples/MGC-170506-10Mo-33-Tirage.pkl',
 '/Users/gilles/Box Sync/2015-Data/Samples/MGC-170506-10Mo-49-Tirage.pkl',
 '/Users/gilles/Box Sync/2015-Data/Samples/MGC-170506-10Mo-22-Tirage.pkl',
 '/Users/gilles/Box Sync/2015-Data/Samples/MGC-170506-10Mo-30-Tirage.pkl',
 '/Users/gilles/Box Sync/2015-Data/Samples/MGC-170506-10Mo-37-Tirage.pkl',
 '/Users/gilles/Box Sync/2015-Data/Samples/MGC-170506-10Mo-01-Tirage.pkl',
 '/Users/gilles/Box Sync/2015-Data/Samples/MGC-170506-10Mo-08-Tirage.pkl',
 '/Users/gilles/Box Sync/2015-Data/Samples/MGC-170506-10Mo-25-Tirage.pkl',
 '/Users/gilles/Box Sync/2015-Data/Samples/MGC-170506-10Mo-42-Tirage.pkl',
 '/Users/gilles/Box Sync/2015-Data/Samples/MGC-170506-10Mo-15-Tirage.pkl',
 '/Users/gilles/Box Sync/2015-Data/Samples/MGC-170506-10Mo-35-Tirage.pkl',
 '/Users/gilles/Box Sync/2015-Data/Samples/MGC-170506-10Mo-41-Tirage.pkl',
 '/Users/gilles/Box Sync/

In [53]:
def lireLexique(nomLexique):
    with open(nomLexique, 'rb') as input:
        lexique=pickle.load(input)
    return lexique

In [54]:
cumulLexique=lireLexique(serieLongitudinale[0])
cumulLexique["tir1"]=0

In [55]:
timeStamp=dt.datetime.now().strftime("%y%m%d-%H%M")
for nTirage,tirage in enumerate(serieLongitudinale):
    lexique=lireLexique(tirage)
    if cumule:
        cumulLexique["tir1"]=cumulLexique["tir1"]+lexique["tir1"]
    else:
        cumulLexique["tir1"]=lexique["tir1"]
    lexique1=cumulLexique[cumulLexique["tir1"]>0]
    nbTokens=lexique1["tir1"].sum()
    nbTypes=lexique1["tir1"].count()
    print "(%d,%d),"%(nbTokens,nbTypes)
    sortieNom=repNomSortie+"Longitudinal-%02d-T%d-F%d-%s.pkl"%(nTirage,nbTokens,nbTypes,timeStamp)
    with open(sortieNom,"wb") as output:
        pickle.dump(lexique1, output, pickle.HIGHEST_PROTOCOL)

(10000000,68060),
(20000000,70812),
(30000000,71822),
(40000000,72390),
(50000000,72811),
(60000000,73109),
(70000000,73339),
(80000000,73519),
(90000000,73680),
(100000000,73821),
(110000000,73937),
(120000000,74049),
(130000000,74148),
(140000000,74247),
(150000000,74351),
(160000000,74420),
(170000000,74512),
(180000000,74621),
(190000000,74685),
(200000000,74749),
